Face Recognition Project with MTCNN, FaceNet, and SVM

Introduction
Ce projet implémente un système de reconnaissance faciale utilisant les outils suivants :

MTCNN pour la détection des visages.
FaceNet pour l'extraction des embeddings.
SVM pour la classification des visages.
Le modèle est entraîné pour reconnaître des célébrités et détecter les visages inconnus en temps réel.



In [19]:
#Importation des bibliothèques nécessaires
import cv2 as cv
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras_facenet import FaceNet
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from mtcnn.mtcnn import MTCNN

In [20]:
# Initialisation du modèle FaceNet
embedder = FaceNet()

1. Fonction get_embedding:
Transformer une image de visage en une représentation numérique unique (embedding), qui peut être utilisée pour identifier ou comparer des visages.


2. Classe FACELOADING
Objectif : Gérer le chargement et le prétraitement des données (images de visages) pour préparer un jeu de données exploitable par le modèle.

3. Fonction extract_face
Objectif : Extraire un visage d'une image donnée.

4. Fonction load_faces
 Charger tous les visages d'un dossier.

5. Fonction load_classes
Charger les données de toutes les classes (célébrités).

In [21]:
# Fonction pour extraire l'embedding d'une image de visage
def get_embedding(face_img):
    face_img = face_img.astype('float32')  # 3D (160x160x3)
    face_img = np.expand_dims(face_img, axis=0)  # 4D (Nonex160x160x3)
    yhat = embedder.embeddings(face_img)
    normalized_embedding = yhat[0] / np.linalg.norm(yhat[0])  # Normalisation L2
    return normalized_embedding

# Classe pour charger les données de visages
class FACELOADING:
    def __init__(self, directory):
        self.directory = directory
        self.target_size = (160, 160)
        self.X = []
        self.Y = []
        self.detector = MTCNN()
    
    def extract_face(self, filename):
        img = cv.imread(filename)
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        detection = self.detector.detect_faces(img)
        if not detection:
            raise ValueError("No face detected in the image.")
        x, y, w, h = detection[0]['box']
        x, y = abs(x), abs(y)
        face = img[y:y+h, x:x+w]
        face_arr = cv.resize(face, self.target_size)
        return face_arr

    def load_faces(self, dir):
        FACES = []
        for im_name in os.listdir(dir):
            try:
                path = os.path.join(dir, im_name)
                single_face = self.extract_face(path)
                FACES.append(single_face)
            except Exception as e:
                print(f"Error processing image {im_name}: {e}")
        return FACES

    def load_classes(self):
        for sub_dir in os.listdir(self.directory):
            path = os.path.join(self.directory, sub_dir)
            FACES = self.load_faces(path)
            labels = [sub_dir for _ in range(len(FACES))]
            print(f"Loaded successfully: {len(labels)} faces for {sub_dir}")
            self.X.extend(FACES)
            self.Y.extend(labels)
        return np.asarray(self.X), np.asarray(self.Y)



Chargement des images


In [22]:
# Chargement des données
faceloading = FACELOADING(r"C:\Users\DELL\Desktop\FaceFinal\dataset")
X, Y = faceloading.load_classes()

Loaded successfully: 10 faces for jenna_ortega
Error processing image 10.jfif: No face detected in the image.
Loaded successfully: 9 faces for robert_downey
Loaded successfully: 10 faces for taylor_swift


Prétraitement des données


In [23]:
# Transformation des images en embeddings
EMBEDDED_X = np.array([get_embedding(img) for img in X])

1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step


In [24]:
# Enregistrement des embeddings et des labels
np.savez_compressed('faces_embeddings_done_3classes.npz', EMBEDDED_X, Y)

In [25]:
# Encodage des labels
encoder = LabelEncoder()
encoder.fit(Y)
Y = encoder.transform(Y)

Entraînement du modèle SVM

In [26]:
# Division des données en ensemble d'entraînement et de test
X_train, X_test, Y_train, Y_test = train_test_split(EMBEDDED_X, Y, shuffle=True, random_state=17)

In [27]:
# Entraînement du modèle SVM
model = SVC(kernel='linear', probability=True)
model.fit(X_train, Y_train)

SVC(kernel='linear', probability=True)

Évaluation du modèle

In [28]:
# Prédiction et évaluation
ypreds_train = model.predict(X_train)
ypreds_test = model.predict(X_test)

print("Train Accuracy:", accuracy_score(Y_train, ypreds_train))
print("Test Accuracy:", accuracy_score(Y_test, ypreds_test))

Train Accuracy: 1.0
Test Accuracy: 1.0


Gestion des visages inconnus

In [29]:
# Ajout de la classe "Unknown"
unknown_threshold = 0.5  # Seuil pour détecter un visage inconnu

In [30]:
def recognize_face(face_embedding):
    probabilities = model.predict_proba([face_embedding])[0]
    max_prob = max(probabilities)
    predicted_label = np.argmax(probabilities)
    
    if max_prob < unknown_threshold:
        return "Unknown", max_prob
    else:
        return encoder.inverse_transform([predicted_label])[0], max_prob

Déploiement en temps réel

In [31]:
# Reconnaissance en temps réel avec la webcam
def recognize_from_camera():
    detector = MTCNN()
    cap = cv.VideoCapture(0)  # Utilisation de la caméra frontale
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame.")
            break
        
        rgb_frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        detections = detector.detect_faces(rgb_frame)
        
        for detection in detections:
            x, y, w, h = detection['box']
            x, y = abs(x), abs(y)
            face = rgb_frame[y:y+h, x:x+w]
            face = cv.resize(face, (160, 160))
            
            # Obtenir l'embedding et reconnaître le visage
            embedding = get_embedding(face)
            name, confidence = recognize_face(embedding)
            
            # Dessiner le rectangle et afficher le nom
            cv.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv.putText(frame, f"{name} ({confidence:.2f})", (x, y-10), cv.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        
        cv.imshow("Face Recognition", frame)
        
        # Quitter avec 'q'
        if cv.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv.destroyAllWindows()




In [18]:
# Démarrage de la reconnaissance en temps réel
recognize_from_camera()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

 Sauvegarde et Chargement du Modèle

In [16]:
import joblib

# Sauvegarde du modèle SVM
os.makedirs('models', exist_ok=True)
joblib.dump(model, 'models/svm_face_recognition_model.pkl')
joblib.dump(encoder, 'models/label_encoder.pkl')



['models/label_encoder.pkl']